# Retail Demand Forecasting – Data Analysis Notebook

In [2]:
import pandas as pd

df = pd.read_csv('../data/raw/train.csv')
df.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [3]:
df.shape

(3000888, 6)

In [4]:
df['date'] = pd.to_datetime(df['date'])

df = df.sort_values(['store_nbr', 'family', 'date']).reset_index(drop=True)

df['sales'] = df['sales'].fillna(0).astype(float)
df['onpromotion'] = df['onpromotion'].fillna(0).astype(int)

df.to_csv('../data/processed/train_clean.csv', index=False)

df.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1782,2013-01-02,1,AUTOMOTIVE,2.0,0
2,3564,2013-01-03,1,AUTOMOTIVE,3.0,0
3,5346,2013-01-04,1,AUTOMOTIVE,3.0,0
4,7128,2013-01-05,1,AUTOMOTIVE,5.0,0
